# Processing tabular data files 

## The Problem

A file holds the names and grades of a class of students for a particular class. The file contents are as follows:

```
name,midterm1,midterm2,final,quiz1,quiz2,quiz3,quiz4
Harry Potter,67,87,45,3,6,8,6
Ronald Weasley,34,65,56,2,3,1,7
Hermione Granger,100,100,100,10,10,10,10
Neville Longbottom,32,12,45,2,3,8,5
```
We will produce a new file with an added column `weighted_average`, and an added row `Average`.

The weighted average for each student is evaluated as follows:
* Midterms have a weight of 20% each.
* Final has a weight of 40%.
* Quizzes have a weight of 5% each.
* Maximum grade is 10 for quizzes, 100 for others.

## The approach

* Open the input file and discard the header line.
* Open the output file for writing
* For each line:
 * Read the grades.
 * Calculate the weighted average.
 * Output the record with the average added.
 * Update the running sums for column averages.

## Process one record

Open the file and read the first record.

In [1]:
fin = open("grades.txt")

In [2]:
line = fin.readline()

In [3]:
line

'name,midterm1,midterm2,final,quiz1,quiz2,quiz3,quiz4\n'

In [4]:
line = fin.readline()

In [5]:
line

'Harry Potter,67,87,45,3,6,8,6\n'

In [28]:
fin.close()

Using this line, we will figure out how to extract the grades and calculate the weighted average.

In [8]:
line.strip()

'Harry Potter,67,87,45,3,6,8,6'

In [9]:
line.strip().split(",")

['Harry Potter', '67', '87', '45', '3', '6', '8', '6']

In [12]:
grades = line.strip().split(",") [1:]  # all but the first element
grades

['67', '87', '45', '3', '6', '8', '6']

The catch is that the grades are strings, and we need to convert them to numbers to do arithmetic on them. We use the `int()` function to do this.

In [13]:
int(grades[0])

67

To convert all elements to int, use the `map()` function.

In [18]:
grades = list(map(int,grades))
grades

[67, 87, 45, 3, 6, 8, 6]

Calculate the weighted average (multiply quizzes by 10 to get a common scale).

In [21]:
wtavg = 0.2*grades[0] + 0.2*grades[1] + 0.4*grades[2] + 0.05*sum(10*grades[3:])
wtavg

60.300000000000004

print the new row.

In [27]:
print(line.strip()+","+str(wtavg))

Harry Potter,67,87,45,3,6,8,6,60.300000000000004


Now we can apply this to all lines.

## Stage 1: Output file with personal averages only

In [1]:
fin = open("grades.txt")
fout = open("gradestmp.txt","w")
fin.readline()  # skip the header line
for line in fin:
    grades = list( map (int, line.strip().split(",") [1:]) ) 
    wtavg = 0.2*grades[0] + 0.2*grades[1] + 0.4*grades[2] + 0.05*sum(10*grades[3:])
    fout.write(line.strip() + "," + str(wtavg) + "\n")
fin.close()
fout.close()

In [2]:
%cat gradestmp.txt

Harry Potter,67,87,45,3,6,8,6,60.300000000000004
Ronald Weasley,34,65,56,2,3,1,7,48.7
Hermione Granger,100,100,100,10,10,10,10,100.0
Neville Longbottom,32,12,45,2,3,8,5,35.8


## Stage 2: Add column averages

We also want to see the class-wide averages for each exam. To that end, we add one record to the bottom that contains the column averages.

First, go over the data file and print the first midterm average.

In [33]:
mt1_sum=0
counter=0
fin = open("grades.txt")
fin.readline()  # skip the header line
for line in fin:
    grades = list( map (int, line.strip().split(",") [1:]) )
    mt1_sum += grades[0]
    counter += 1
print("Midterm 1 average =",mt1_sum/counter)

Midterm 1 average = 58.25


Check:

In [34]:
(67+34+100+32)/4

58.25

Now apply to all columns. However, instead of defining seven different accumulator variables, we define one list with seven elements.

In [36]:
list(enumerate(["a","b",334,"asdfg"]))

[(0, 'a'), (1, 'b'), (2, 334), (3, 'asdfg')]

In [40]:
sums =[0,0,0,0,0,0,0]
fin = open("grades.txt")
fin.readline()  # skip the header line
counter=0
for line in fin:
    grades = list( map (int, line.strip().split(",") [1:]) )
    for i,g in enumerate(grades):
        sums[i] += g
    counter += 1
# divide by the student count
for i in range(len(sums)):
    sums[i] /= counter
fin.close()
averages = ",".join(map(str,sums))
print("Average,"+averages)

Average,58.25,66.0,61.5,4.25,5.5,6.75,7.0


Finally, combine the first and second stages; output to a file.

In [3]:
fin = open("grades.txt")
fout = open("gradesaverages.txt","w")
fin.readline()  # skip the header line
sums = [0,0,0,0,0,0,0,0] # One more element, for wtavg
counter = 0
for line in fin:
    grades = list( map (int, line.strip().split(",") [1:]) )
    wtavg = 0.2*grades[0] + 0.2*grades[1] + 0.4*grades[2] + 0.05*sum(10*grades[3:])
    fout.write(line.strip() + "," + str(wtavg) + "\n")
    # Column sums:
    for i,g in enumerate(grades):
        sums[i] += g
    sums[-1] += wtavg
    counter += 1
fin.close()

# Add the extra line
for i in range(len(sums)):
    sums[i] /= counter
averages = ",".join(map(str,sums))
fout.write("Average,"+averages+"\n")
fout.close()

In [4]:
%cat gradesaverages.txt

Harry Potter,67,87,45,3,6,8,6,60.300000000000004
Ronald Weasley,34,65,56,2,3,1,7,48.7
Hermione Granger,100,100,100,10,10,10,10,100.0
Neville Longbottom,32,12,45,2,3,8,5,35.8
Average,58.25,66.0,61.5,4.25,5.5,6.75,7.0,61.2
